In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/random'
folder = 'sp_matrix_tag_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [3]:
predictions[:5]

,uid,tid,rating
0,0,2,0.070515
1,0,14,0.022937
2,0,15,0.147609
3,0,20,0.038996
4,0,21,0.023123


In [4]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [5]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5
6
7
8
9
10
20


In [6]:
num_user = len(predictions['uid'].unique())
num_user

218

In [7]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

In [8]:
evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.16192660550458718


In [9]:
groups = np.loadtxt(os.path.join(dir_, 'groups', 'random' ,'2.csv'), delimiter=',')
group_size = len(groups[0])
groups[0]

array([37., 29.])

In [10]:
n = 20
top_n_lists_g = []
    
for group in tqdm(groups):
    u1 = predictions[predictions['uid'] == group[0]]
    u2 = predictions[predictions['uid'] == group[1]]

    m = pd.merge(u1, u2, on=['tid'], how='inner', suffixes=['_u1','_u2'])
    m = m[['tid', 'rating_u1','rating_u2']]
    
    m['rating'] = (m['rating_u1'] + m['rating_u2'])/2
    m = m[['tid', 'rating']]
    m = m.sort_values(by=['rating'],  ascending=False)
    m = m[:n]
    top_n_list = []
    for _, row in m.iterrows():
        top_n_list.append(row[0])
    top_n_lists_g.append(top_n_list)
    
satisfication = 0
for i in tqdm(range(len(groups))):
    group = groups[i]
    top_n_list = top_n_lists_g[i]
    precision = 0
    for j in top_n_list:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            rating = 0.0
            high_rating = 0
            if len(t1) > 0 and t1.iloc[0]['rating'] > rating:
                high_rating +=1
            if len(t2) > 0 and t2.iloc[0]['rating'] > rating:
                high_rating +=1
            if high_rating > 0:
                precision += 1
    satisfication += precision / n
print(satisfication/(len(groups)))


0.2816513761467892


In [28]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

In [29]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        
                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)


In [30]:
for i in precision_4:
    print(i)
print('------------------')

for i in precision_20:
    print(i)
print('------------------')

for i in precision_50:
    print(i)
print('------------------')

for i in recall_4:
    print(i)
print('------------------')

for i in recall_20:
    print(i)
print('------------------')

for i in recall_50:
    print(i)

[0.19036697247706422, 0.17476851851851852, 0.18287037037037038, 0.15813953488372093, 0.1724537037037037, 0.14516129032258066, 0.14351851851851852, 0.1550925925925926, 0.1392857142857143, 0.115]
------------------
[0.1488532110091744, 0.13171296296296311, 0.12662037037037052, 0.12325581395348842, 0.12013888888888896, 0.11705069124423972, 0.11342592592592603, 0.10856481481481499, 0.10809523809523819, 0.09775000000000006]
------------------
[0.12220183486238516, 0.11240740740740726, 0.10592592592592569, 0.10306976744186035, 0.10037037037037023, 0.10101382488479252, 0.09249999999999986, 0.09055555555555543, 0.09438095238095225, 0.08529999999999992]
------------------
[0.0021881481706991887, 0.001897628820088124, 0.0020146460527965303, 0.00178986655888081, 0.0018671140477664965, 0.0013850151248872183, 0.0013918312558256023, 0.0019682417244809482, 0.001398476769208736, 0.001498713923827305]
------------------
[0.008103237373957956, 0.00703160203024336, 0.007178062636648887, 0.006841795784312

In [54]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [55]:
nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [56]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

[5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5
 4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.
 4.   4.   4.   4.   3.5  3.5  3.5  3.5 ]
[5.   5.   5.   4.   4.   4.   4.   4.   3.5  3.5  3.5  3.5  3.5  3.
 3.   3.   3.   3.   2.75 2.5  2.5  2.5  2.25 2.25 2.25 2.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.
 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75
 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5
 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5 ]
[5.   5.   4.75 4.75 4.75 4.25 4.25 4.25 4.25 3.5  3.5  3.25 3.25 3.25
 2.75 1.75 1.75 1.75 1.75 1.75 1.75 1.75 1.75 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5

[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.
 5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5
 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  3.
 3.   3.   3.   3.   3.   3.   3.   3.  ]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5.]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5
 4.5  4.25 4.25 4.25 4.25 4.25 4.25 4.25]
[5.   5.   5.   4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25
 4.25 4.25 4.25 

[5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.5  4.5  4.5  4.5  4.5  4.5  4.   4.   4.   4.   4.
 4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   2.75 2.75
 2.75 2.75 2.75 2.75 2.75 2.75 2.75 2.75]
[5.   5.   5.   5.   4.75 4.75 4.75 4.75 4.75 4.   4.   4.   4.   4.
 4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.
 4.   0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.
 5.   4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.75 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5
 4.5  4.5  4.5  4.5  4.25 4.25 4.25 4.25]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.75 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5
 4.5  4.5  4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25
 4.25 4.25 4.   4.   4.   4.   4

[5.   5.   4.75 4.75 4.5  4.25 4.25 4.25 4.25 4.   4.   3.75 3.75 3.5
 3.5  3.5  3.5  3.5  3.25 3.25 3.25 3.25 2.75 2.75 2.75 2.25 2.25 2.25
 2.25 2.25 2.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
[5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75 4.25 4.25 4.25
 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.25
 4.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
[5.   5.   5.   4.75 4.75 4.5  4.5  4.5  4.5  4.25 4.25 4.25 4.25 4.25
 4.25 4.   4.   4.   4.   4.   4.   3.5  3.5  3.5  3.5  3.5  3.5  3.25
 3.   3.   3.   3.   3.   2.5  2.5  2.5  2.5  2.5  2.25 2.25 2.25 2.25
 2.25 2.25 2.   2.   2.   2.   2.   1.5 ]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5  4.5
 4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.   4.
 4.   4.   4.   3.   3.   3.   

[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5.]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75
 4.75 4.5  4.5  4.5  4.5  4.25 4.25 4.25 4.25 4.25 4.25 4.25 4.   4.
 4.   4.   4.   4.   4.   3.75 3.75 3.75 3.75 3.75 3.75 3.75 3.75 3.75
 3.5  3.5  3.5  3.5  3.5  3.5  3.5  3.5 ]
[4.75 4.75 4.75 4.75 3.75 3.75 3.75 3.75 3.75 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75
 4.5  4.5  4.5  4.5  4.5  4.5  4.   4.   4.   4.   4.   4.   4.   4.
 4.   4.   4.   0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
[5.   5.   5.   5.   0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.2

...

[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.75]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.
 4.75 4.75 4.75 4.75 4.75 4.75]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[5.   0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25]
[5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   4.75
 4.75 4.75 4.75 4.75 4.75 4.75]
[5.   5.   5.   5.   5.   5.   5.   4.75 4.75 4.75 4.75 4.75 4.75 4.75
 4.75 4.75 4.75 4.75 4.75 4.75]
[5.   5.   5.   4.75 4.75 4.75 4.75 4.75 4.75 4.   4.   4.   4.   4.
 4.   4.   4.   4.   4.   4.  ]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[5.   5.   5.   5.   5.   5.   5.   5.   4.25 4.25 4.25 4.25 4.25 4.25
 4.25 4.25 4.25 4.25 4.25 4.25]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[5.   5.   5.   4.75 4.75 4.75 4.75

[5.  4.5 4.5 4.5]
[5. 5. 5. 5.]
[5.   5.   5.   4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   3.75 3.75 3.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[0.25 0.25 0.25 0.25]
[5. 5. 5. 5.]
[5.  5.  5.  4.5]
[5. 5. 5. 5.]
[5.   5.   5.   4.75]
[5.   4.25 4.25 4.25]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   4.25 4.25 3.75]
[4.75 4.75 4.75 4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 4.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.

[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   5.   4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[4.75 4.75 4.75 4.  ]
[5. 5. 5. 5.]
[4.75 4.75 4.75 4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   0.25 0.25 0.25]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.  5.  4.5 4.5]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[0.25 0.25 0.25 0.25]
[5. 5. 5. 5.]
[4.75 4.75 4.75 4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   5.   4.25]
[5. 5. 5. 5.]
[5.   3.75 3.75 3.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   5.   4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[4.75 4.75 4.7

[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   4.25 4.25 3.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[0.25 0.25 0.25 0.25]
[5. 5. 5. 5.]
[5.   5.   0.25 0.25]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[4.75 4.75 4.75 4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 4.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[0.25 0.25 0.25 0.25]
[5.   5.   5.   4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.  5.  4.5 4.5]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   4.25 4.25 3.5 ]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[0.25 0.25 0.25 0.25]
[5. 5. 5. 5.]
[5.   0.25 0.25 0.25]
[5. 5. 5. 5.]
[5.   4.25 4.25 4.25]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5.

[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 4.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   4.75 4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   4.75 4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   0.25 0.25]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[4.75 4.75 4.75 4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 4.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[4.5  4.   3.25 3.25]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   0.25 0.25 0.25]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   5.   4.75]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5. 5. 5. 5.]
[5.   5.   5.   4.75

In [57]:
for i in nDCG_4:
    print(i)
print('------------------')

for i in nDCG_20:
    print(i)
print('------------------')

for i in nDCG_50:
    print(i)

[0.08321207574110338, 0.08051860722469223, 0.07723919446003817, 0.07732555539848887, 0.07829999487215487, 0.06291921329126685, 0.06335836573156399, 0.06505391651118547, 0.06385631767560186, 0.04210833073400726]
------------------
[0.06947824903286301, 0.06038965656190081, 0.05830290747258697, 0.05978121156734599, 0.05846030979025819, 0.05027005652436385, 0.05284840471860665, 0.04992834671838403, 0.05046354858898879, 0.03895441564722248]
------------------
[0.06003098313614616, 0.05416648244544796, 0.05214413803704013, 0.05253625175315135, 0.051194656622487816, 0.04659657326706828, 0.04618878201406208, 0.0451414155646256, 0.04598538250680677, 0.03804079203154635]
